# Импорт библиотек

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd 
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
# import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score

# Предобработка данных

Для выполнения задания возьмем датасет *NBA 2k20 player dataset* и изучим основную информацию о нем

In [ ]:
df = pd.read_csv('nba2k-full.csv')
df.head()

Информация о датасете

In [ ]:
df.shape

In [ ]:
df.info()

Устраним пропуски

In [ ]:
df['team'] = df['team'].fillna('not specified')
df['college'] = df['college'].fillna('not specified')

Преобразуем данные в столбцах: уберем знак доллар в зарплатах, рост и вес приведем к чесленному формату, измеряя рост в метрах, а вес в кг

In [ ]:
for i in range(df.shape[0]):         # обработка даты рождения
    s = df['b_day'][i]
    if s[6] == '0' or s[6] == '1' or s[6] == '2':
        s = s[0:6] + '20' + s[6:8]
    else:
        s = s[0:6] + '19' + s[6:8]
    df['b_day'][i] = s
    s = ''
df['b_day'] = pd.to_datetime(df['b_day'], format='%m/%d/%Y')

df['height_meters'] = float(0)  #обработка роста
for i in range(len(df['height'])):
    s = df['height'][i].split('/')
    df['height_meters'][i] = float(s[1])
    s = ''

df['weight_kg'] = float(0)  #обработка веса
for i in range(len(df['weight'])):
    s = df['weight'][i].split('/')
    s1 = s[1].split()
    df['weight_kg'][i] = float(s1[0])
    s = ''
    s1 = ''

df['salary_dollars'] = int(0)  #обработка зарплаты
for i in range(len(df['salary'])):
    s = df['salary'][i]
    df['salary_dollars'][i] = int(s[1:])
    s = ''    
    
df.info()

# Дискретизируем признак

Разобъем зарплаты на равные группы и создадим столбец *salary_group*, где группы будут распределены следующим образом:
- 1: низкая
- 2: высокая

In [ ]:
df['salary_group'] = pd.qcut(df['salary_dollars'], 
                             2, 
                             [1,2])

# Выделим признаки для предсказания

Что предсказываем: группу зараплаты

Оставим в датафрейме: рейтинг, рост, вес, год драфта

In [ ]:
X = df.copy(deep=True)
X = X.drop(['full_name', 'jersey', 'team', 
                   'position', 'b_day','height', 'salary', 
                   'country', 'draft_round', 'draft_peak', 
                   'college', 'salary_group', 'weight', 'version'], axis=1).values.tolist()
y = list(df['salary_group'])

Обучение модели, сперва разобъем на тренировочную и валидационную части

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Выберем модель случайного леса

In [ ]:
rf_model = RandomForestClassifier() 
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Проверим нормальность количественного признака

Проверим рост на нормальность распределения

In [ ]:
st.normaltest(df['height_meters'])

Построим гистограмму

In [ ]:
df['height_meters'].hist()

Данные немного скошены, но в целом имеют вид нормального распределения

# Метрики проверки качества предсказаний модели

In [ ]:
def print_all_metrics(y_true, y_pred, title='Метрики классификации'):
    print(title)
    print('\tAccuracy: {:.5f}'.format(accuracy_score(y_true, y_pred)))
    print('\tPrecision: {:.5f}'.format(precision_score(y_true, y_pred)))
    print('\tRecall: {:.5f}'.format(recall_score(y_true, y_pred)))
    print('\tF1: {:.5f}'.format(f1_score(y_true, y_pred)))

print_all_metrics(
    y_test,
    rf_predictions,
    title='Метрики для модели случайного леса:'
)